# Exericse 05: Gauss-Newton minimization
In this exercise you will implement direct image alignment as Gauss-Newton minimization on $ SE(3) $.
For details see chapter 2 of the lecture slides as well as the theoretical exercise sheets 8 and 9.

In [5]:
import os
import numpy as np
from exercise_code import load_dataset, down_scale, derive_jac_and_residual, update_step,  se3Log, se3Exp
%load_ext autoreload
%autoreload 2

img1, img2, depth1, depth2, K = load_dataset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The algorithm contains the following steps:
1. Downscale the image, depth and camera intrinsics. The larger the level is, the smaller the image size, e.g., $640 \times 480 (level=1)$,  $320 \times 240 (level=2)$.
2. Compute the Jacobian and also the residual.
3. Perform a Gauss-Newton Step to update $\xi$.

You need to implement the following functions:

In `exercise_code/utils.py`, implement the `down_scale` function using the theoretical results from exercise 08.

In `exercise_code/utils.py`, implement `se3Log` and `se3Exp` functions.

In `exericse_code/Gauss_Newton.py`, finish the function `derive_jac_and_residual` and `update_step`.

This is a minimal setup to test your implementation (Fast):

In [23]:
level = 6  # we start from the coarsest level (i.e. the resolution is the lowest)
xi = np.zeros(6) # initialize the camera motion
while level > 0:
    img_ref, depth_ref, K_ref = down_scale(img1, depth1, K, level)
    img_tag, depth_tag = down_scale(img2, depth2, K, level)[:2]

    errorLast = 1e10
    # we iterate 20 times at most.
    for i in range(20):
        jac, residual = derive_jac_and_residual(img_ref, depth_ref, img_tag, xi, K_ref)
        xi = update_step(jac, residual, xi)

        error = np.mean(residual ** 2)
        print(f"level {level}, iter {i}, MSE={error:.2e}")
        if error / errorLast > 0.995:
            break
        errorLast = error
    level -= 1
print("Final xi:", xi)

level 6, iter 0, MSE=1.63e-02
level 6, iter 1, MSE=4.25e-03
level 6, iter 2, MSE=1.51e-02
level 5, iter 0, MSE=3.63e-02
level 5, iter 1, MSE=4.13e-02
level 4, iter 0, MSE=1.86e-02
level 4, iter 1, MSE=4.43e-02
level 3, iter 0, MSE=2.70e-03
level 3, iter 1, MSE=2.37e-02
level 2, iter 0, MSE=2.81e-02
level 2, iter 1, MSE=3.70e-02
level 1, iter 0, MSE=3.32e-02
level 1, iter 1, MSE=1.13e-01
Final xi: [-0.0032736   0.00045871  0.03562489 -0.03254532 -0.01719981 -0.00165318]


(Optional) Below is the same procedure, just with additional visualizations to help build your intuition. (Slow) This shows reference points projected into the target image at each iteration, allowing you to visually track the motion estimation process and how well the reference points align over time.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

max_level = 6
xi = np.zeros(6)  # initial camera motion

# run optimization for each level
frames    = []
for level in range(max_level, 0, -1):
    # print(f"\n--- Solving at pyramid level {lvl} ---")
    # xi = run_level(lvl, xi)

    img_ref, depth_ref, K_ref = down_scale(img1, depth1, K, level)
    img_tag, depth_tag, _    = down_scale(img2, depth2, K, level)

    # sparse grid for visualization later
    stride = 2 ** (max_level - level + 1)
    ys = np.arange(0, img_ref.shape[0], stride)
    xs = np.arange(0, img_ref.shape[1], stride)
    grid_x, grid_y = np.meshgrid(xs, ys)
    sx, sy = grid_x.ravel(), grid_y.ravel()
    valid = depth_ref[sy, sx] > 0
    sx, sy = sx[valid], sy[valid]

    errorLast = 1e10

    for it in range(20):
        # compute Jacobian & residual 
        jac, residual = derive_jac_and_residual(
            img_ref, depth_ref,    # reference
            img_tag,               # target image
            xi,              # current guess
            K_ref                  # correct intrinsics for this level
        )
        # update twist
        xi = update_step(jac, residual, xi)

        error = np.mean(residual**2)
        print(f"level {level}, iter {it+1}, MSE={error:.2e}")

        #early-stop check
        if error / errorLast > 0.995:
            break
        errorLast = error

        # record frame for sparse points
        pts = np.vstack([sx, sy, np.ones_like(sx)])
        p3d = np.linalg.inv(K_ref) @ (pts * depth_ref[sy, sx])
        T   = se3Exp(xi)
        R, t = T[:3,:3], T[:3,3]
        p3d_t  = R @ p3d + t[:,None]
        proj   = K_ref @ p3d_t
        x2 = proj[0]/proj[2]; y2 = proj[1]/proj[2]
        vis = p3d_t[2] > 0

        frames.append({
            'x1': sx,
            'y1': sy,
            'x2': x2[vis],
            'y2': y2[vis],
            'error': error,
            'iter':  it+1,
            'img_ref': img_ref,
            'img_tag': img_tag,
            'level': level
        })
print("Generating animation with", len(frames), "frames...")

# Vis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))


def update_frame(i):
    f = frames[i]
    fig.suptitle(f"Level {f['level']} • Iteration {f['iter']} • MSE={f['error']:.2e}", fontsize=14)

    ax1.clear()
    ax1.imshow(f['img_ref'], cmap='gray')
    ax1.scatter(f['x1'], f['y1'], s=20, facecolors='none', edgecolors='b')
    ax1.set_title(f'reference samples')
    ax1.axis('off')

    ax2.clear()
    ax2.imshow(f['img_tag'], cmap='gray')
    sc2 = ax2.scatter([], [], s=20, facecolors='none', edgecolors='r')
    sc2.set_offsets(np.column_stack([f['x2'], f['y2']]))
    ax2.set_title(f'target samples')
    ax2.axis('off')

ani = FuncAnimation(fig, update_frame,
                    frames=len(frames),
                    interval=200,
                    repeat_delay=1000)
display(HTML(ani.to_jshtml()))
plt.close(fig)

level 6, iter 1, MSE=1.63e-02
level 6, iter 2, MSE=3.91e-03
level 6, iter 3, MSE=3.15e-02
level 5, iter 1, MSE=2.82e-03
level 5, iter 2, MSE=6.11e-02
level 4, iter 1, MSE=8.10e-03
level 4, iter 2, MSE=4.53e-02
level 3, iter 1, MSE=8.47e-02
level 3, iter 2, MSE=4.27e-02
level 3, iter 3, MSE=8.44e-02
level 2, iter 1, MSE=5.93e-02
level 2, iter 2, MSE=8.23e-02
level 1, iter 1, MSE=6.87e-02
level 1, iter 2, MSE=1.18e-01
Generating animation with 8 frames...


In [28]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/exercise05')

relevant folders: ['exercise_code']
notebooks files: ['gauss-newton_method.ipynb']
Adding folder exercise_code
Adding notebook gauss-newton_method.ipynb
Zipping successful! Zip is stored under: /Users/chenshien/Documents/Notes/2025 Summer (TUM)/Computer Vision II/cv2mvg/output/exercise05.zip
